In [26]:
import os
import django

# Set the DJANGO_SETTINGS_MODULE to your project's settings module
os.environ['DJANGO_SETTINGS_MODULE'] = 'QueryWorkshop25.settings'

# Initialize Django
django.setup()

In [27]:
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

In [28]:
from django.db import connection, reset_queries
from books.models import *

reset_queries()


class query_count_manager:
    """
    A context manager to count the number of queries executed within its scope.
    """

    def __init__(self, time=False):
        self.time = time

    def __enter__(self):
        self.initial_query_count = len(connection.queries)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.final_query_count = len(connection.queries)
        self.query_count = self.final_query_count - self.initial_query_count
        print(f"Number of queries executed: {self.query_count}")

        if self.time:
            self.total_time = sum(float(query['time']) for query in connection.queries[self.initial_query_count:])
            print(f"Approximate time taken for all queries: {self.total_time} seconds")


# When are Querysets evaluated?
Querysets are lazy, meaning they are not evaluated until you actually need the data. This allows for optimizations and avoids unnecessary database queries. Django will also cache the results, meaning that accessing attributes multiple times will not hit the database againg.

In [29]:
book = BookData.objects.first()
book.title  # This will hit the database
book.title  # This will not hit the database again, as the queryset is cached

'Figure factor kitchen.'

Generally whenever you use a callable attribute you will hit the database, for example:

In [30]:
book = BookData.objects.get(id=1)
book.authors.all()  # query performed
book.authors.all()  # query performed again

<QuerySet [<Author: Anne Smith>, <Author: Ashley Bryant>, <Author: Tyler Wilson>, <Author: Alyssa Salazar>, <Author: Jerry Ryan>, <Author: Jennifer Sanchez>]>

## Operations that will force queryset evaluation
1. Iteration (e.g., in a for loop).
2. Slicing (e.g., `[10]`).
3. Pickling/Serializing.
4. repr() or str() conversion.
5. len() function.
6. Boolean evaluation (e.g., `if queryset:`).
7. Converting to a list (e.g., `list(queryset)`).


# 1. Iteration

In [31]:
reset_queries()

books = BookData.objects.all()  # This creates a queryset but does not hit the database yet
print("query count after queryset creation:", len(connection.queries))

book_list = [b for b in books]  # The queryset is not evaluated until we iterate over it
print("query count after iteration:", len(connection.queries))

# Django caches the queryset, so subsequent iterations will not hit the database again
book_list_2 = [b for b in books]  # This will not hit the database again
print("query count after second iteration:", len(connection.queries))

query count after queryset creation: 0
query count after iteration: 1
query count after second iteration: 1


# 2. Slicing
Slicing a queryset will not hit the database until you actually access the data. This allows you to limit the number of records fetched.


In [32]:
reset_queries()
all_books = BookData.objects.all()

books = all_books[:5]  # This does not hit the database yet
print("query count after slicing:", len(connection.queries))

# Accessing a single entry of the data will evaluate the queryset
book = books[0]
print("query count after accessing data:", len(connection.queries))

# using the step argument in slicing will however hit the database
stepped_books = all_books[::2]  # This will hit the database
print("query count after stepped slicing:", len(connection.queries))

query count after slicing: 0
query count after accessing data: 1
query count after stepped slicing: 2


# 3. Serializing/Cacheing
Serializing or cacheing a queryset will evaluate it, hitting the database to fetch the data.

In [33]:
from django.core.cache import cache

with query_count_manager():
    books = BookData.objects.all()
    cache.set('books', books, 0)  # This will hit the database

Number of queries executed: 1


# 4. repr() or str() conversion
When you convert a queryset to a string or use `repr()`, it will evaluate the queryset and hit the database. This usually happens when you print the queryset or use it in a template.

In [34]:
with query_count_manager():
    books = BookData.objects.all()[:10]
    print(books)


<QuerySet [<BookData: Figure factor kitchen.>, <BookData: Politics election dog.>, <BookData: Art born.>, <BookData: Age them include already.>, <BookData: Role.>, <BookData: Clear.>, <BookData: Because.>, <BookData: Activity interview near.>, <BookData: Employee yard account.>, <BookData: Quality house.>]>
Number of queries executed: 1


# 5. len()
Using len will evaluate a queryset and hit the database to count the number of records.

In [35]:
with query_count_manager():
    books = BookData.objects.all()
    len(books)  # This will hit the database to count the records

# There is a better way to do this to using `count()` as the db is not sending all records back.
number_books = BookData.objects.count()

Number of queries executed: 1


# 6. Boolean evaluation
Using a queryset in a boolean context will evaluate it and hit the database.


In [36]:
with query_count_manager():
    books = BookData.objects.all()
    if books:  # This will hit the database to check if there are any records
        print("There are books in the database")

# There is a better way to do this using `exists()` as it check for existence without counting or fetching all records.
number_books_exist = BookData.objects.exists()

There are books in the database
Number of queries executed: 1


# 7. Converting to a list

In [37]:
with query_count_manager():
    books = BookData.objects.all()
    book_list = list(books)  # This will hit the database to fetch all records
    print(f"Number of books fetched: {len(book_list)}")

Number of books fetched: 5000
Number of queries executed: 1


# N+1 Queries Problem

The N+1 queries problem occurs when a query is made for each item in a queryset, leading to excess database accesses. For each object you are hitting the database an additional time to fetch the title of the book. This can be mitigated by using `select_related` or `prefetch_related`.

In [38]:
# Bad N+1 query example
with query_count_manager():
    # This will execute the query and count the number of queries made
    books = Book.objects.all()
    for book in books[:1000]:
        _ = book.book_data.title


Number of queries executed: 1001


# Select Related
Using `select_related` allows you to fetch related objects in a single query, reducing the number of queries executed. This can only be used for single-valued relationships (ForeignKey, OneToOneField) and not ManyToMany relationships.

In [39]:
with query_count_manager():
    books = Book.objects.all().select_related('book_data')
    for book in books[:1000]:
        _ = book.book_data.title

Number of queries executed: 1


# Prefetch Related
Using `prefetch_related` allows you to fetch related objects in a separate query, which is then joined together in Python. This is useful for ManyToMany relationships or when you want to fetch multiple related objects to avoid everything being done in one very complex query, which can take longer.

In [40]:
with query_count_manager():
    book_data = Book.objects.all().prefetch_related('library')

    for book in book_data[:1000]:
        _ = book.library.name


Number of queries executed: 2


# Get the DB to do the work rather than Python

There are many operation that are done more efficiently by the database rather than in Python. You can also take advantage of the fact that the database is designed to handle large amounts of data efficiently. Here are some examples:

## 1. Filtering
Filtering data in the database is more efficient than filtering it in Python. Use the `filter()` and `exclude` methods to retrieve only the records you need.

In [41]:
books = list(BookData.objects.filter(page_count__gt=100))  # Get all books with more than 100 pages

In [42]:
all_books = BookData.objects.all()
books = []
for book in all_books:
    if book.page_count > 100:
        books.append(book)  # This is less efficient than filtering in the database

## 2. F Expressions
F Expression can be used to directly reference fields in the database, allowing you to make changes without having to even load the objects into memory. This is particularly useful for bulk updates or calculations.

In [43]:
from django.db.models import F

with query_count_manager():
    # I didn't like the ending to any of the books, so I decided to write my own ending instead
    # Increase the page count of all books by 10
    BookData.objects.update(page_count=F('page_count') + 10)

Number of queries executed: 1


In [44]:
# That was much faster than iterating over all books and updating them one by one
with query_count_manager():
    books = BookData.objects.all()
    for book in books:
        book.page_count -= 10  # The library made me do this :(
        book.save()

Number of queries executed: 5001


In [45]:
# F Expressions can also be used to filter data based on calculations
# Which books were written before their authors were born?
BookData.objects.filter(authors__birth_date__gt=F('published_date')).distinct()

<QuerySet [<BookData: Figure factor kitchen.>, <BookData: Politics election dog.>, <BookData: Art born.>, <BookData: Age them include already.>, <BookData: Role.>, <BookData: Clear.>, <BookData: Because.>, <BookData: Activity interview near.>, <BookData: Employee yard account.>, <BookData: Quality house.>, <BookData: Road degree.>, <BookData: Appear program get.>, <BookData: Daughter local past.>, <BookData: Deal certain.>, <BookData: Cell focus her.>, <BookData: There protect bank benefit.>, <BookData: Sometimes thank data.>, <BookData: Rise born.>, <BookData: Deal half look.>, <BookData: Especially soldier.>, '...(remaining elements truncated)...']>

# 3. Aggregation/Annotation
Aggregation allows you to perform calculations on your data directly in the database, such as counting, summing, averaging, etc. This is generally more efficient than doing these calculations in Python.


In [46]:
len(BookData.objects.all())

5000

In [47]:
BookData.objects.all().count()

5000

In [48]:
# Annotation allows you to add calculated fields to your queryset, which can be used for further filtering or ordering.
from django.db.models import Count

# Count the number of books for each author
Author.objects.annotate(book_count=Count('books')).values_list("name", "book_count")

<QuerySet [('Brian Perry', 23), ('Lauren Hernandez', 29), ('Jessica Vance', 21), ('Mr. Michael Johnson', 28), ('Christopher Schneider', 23), ('Sara Dixon', 35), ('Emma Koch', 27), ('Brian Roberts', 18), ('Heather Garner', 27), ('Kimberly Ortiz', 21), ('Pamela Molina', 22), ('Donald Robbins', 26), ('Kathryn Newton', 27), ('Ernest Payne', 30), ('Jeffrey Brown', 30), ('Zachary Vega', 27), ('Michael Taylor', 25), ('Sara Mills', 24), ('Roy Smith', 30), ('Paige Gibson', 23), '...(remaining elements truncated)...']>

# Indexing
Indexing is a powerful way to speed up database queries. It allows the database to quickly locate and retrieve data without scanning the entire table. You can create indexes on fields that are frequently used in filters or joins. But be careful, as too many indexes can slow down write operations, as the index needs to be recalculated when new data is added.

```python

class Library(models.Model):
    address = models.CharField(max_length=300, db_index=True)  # Adding an index to the address field
    name = models.CharField(max_length=200)

    class Meta:
        indexes = [
            models.Index(fields=['name'], name='library_name_idx'), # Custom index on the name field
        ]
```
using the unique constraint on the ISBN field will automatically create an index on that field, which can speed up lookups.

In [49]:
last_book_data = BookData.objects.last()

In [51]:
BookData.objects.get(isbn=last_book_data.isbn).isbn  # This will use the index on the isbn field to quickly locate the book

'978-0-367-644'

In [52]:
BookData.objects.get(
    isbn_not_unique=last_book_data.isbn_not_unique).isbn_not_unique  # This will not use the index, as the field is not unique

'978-0-04-9557'

# Only Fetch What You Need
When working with large datasets, it's important to only fetch the data you need. This can be done using the `only()` and `defer()` methods to limit the fields fetched from the database. `only()` will fetch only the specified fields, while `defer()` will fetch all fields except the specified ones. This can significantly reduce the amount of data transferred from the database to your application. If you do use a field that was deferred, it will hit the database to fetch that field.

In [53]:
# Only
with query_count_manager():
    books = BookData.objects.only('title', 'isbn')[:10]  # Only fetch the title and isbn fields
    for book in books:
        _ = book.title, book.isbn  # This will not hit the database again, as the fields are already fetched

Number of queries executed: 1


In [ ]:
with query_count_manager():
    books = BookData.objects.only('title', 'isbn')[:10]  # Fetch all fields
    for book in books:
        _ = book.title, book.page_count  # This will hit the database again, as the page_count field was not included

# Values and Values List
Using `values()` or `values_list()` allows you to fetch only the fields you need, returning dictionaries or tuples instead of model instances. This can be more efficient when you don't need the full model functionality.

In [ ]:
from pympler import asizeof

all_books = list(BookData.objects.all())

# profile to see memory usage
print(f"Memory usage before values: {asizeof.asizeof(all_books)} bytes")

In [ ]:
all_books_values = list(BookData.objects.values("title"))
print(f"Memory usage after values: {asizeof.asizeof(all_books_values)} bytes")

# Iterator
Using `iterator()` can be more memory-efficient when dealing with large querysets. It fetches the data in chunks instead of loading the entire queryset into memory at once. It will also not cache the queryset, so you will hit the database every time you access the data from scratch. That means that the data you've already looped through will be discarded, reducing the memory cost of the queryset.


In [ ]:
with query_count_manager():
    books = BookData.objects.all().iterator(chunk_size=200)  # Fetch data in chunks
    for book in books:
        _ = book.title  # This will hit the database for each chunk, but not for each book

        # the books iterator is now empty, as it has been exhausted
        # I'd need to fetch the data again to access it

    books = BookData.objects.all().iterator(chunk_size=200)  # Fetch data in chunks
    for book in books:
        _ = book.title


# Bulk Methods
Bulk methods allow you to perform operations on multiple records at once, which can be more efficient than doing them one by one. This is particularly useful for creating or updating large numbers of records.

- Bulk Create: `bulk_create()`
- Bulk Update: `bulk_update()`
- Bulk Delete: `bulk_delete()`

In [ ]:
new_book = BookData(title="New Book", isbn="978-3-16-148410-0", published_date="2023-10-01", page_count=100)
second_new_book = BookData(title="Second New Book", isbn="978-3-16-148410-1", published_date="2023-10-02", page_count=200)


In [ ]:

with query_count_manager():
    # Using bulk_create to create multiple records at once
    BookData.objects.bulk_create([new_book, second_new_book])  # This will hit the database once to create both records


In [ ]:
with query_count_manager():
    # Using bulk_update to update multiple records at once

    new_book.title = "Updated New Book"
    second_new_book.page_count = 210  # I wrote a few more pages
    BookData.objects.bulk_update([new_book, second_new_book], ["title", "page_count"])  # This will hit the database once to update both records

In [ ]:
with query_count_manager():
    # Using bulk_delete to delete multiple records at once
    BookData.objects.filter(id__in=[new_book.id, second_new_book.id]).delete() # This will hit the database once to delete both records